In [30]:
%matplotlib inline
import networkx as nx
import matplotlib.pyplot as plt
import random
import pandas as pd
import numpy as np
import multiprocessing
from joblib import Parallel, delayed
import copy, os, csv, time
import fnmatch

In [15]:
g_filename = None
g_case_list = None
g_year_map = None

def get_file_obj(filename):
    return open(filename, 'r')

def get_year_map(filename):
    
    global g_filename
    global g_case_list
    global g_year_map
    
    if (g_filename is not None) and (g_filename == filename):
        if (g_case_list is not None) and (g_year_map is not None):
            return g_year_map, g_case_list
        
    
    f = get_file_obj(filename)
    
    year_map = {}
    case_list = []
    
    for line in f:
        year, from_case, to_case = line[:-1].split(',')
        
        if from_case not in year:
            year_map[from_case] = int(year)
            case_list.append(from_case)
        
    f.close()
    
    g_filename = filename
    g_year_map = year_map
    g_case_list = case_list
    
    
    
    return year_map, case_list


def get_graph(filename, b_enforce_random = False):
    
    if b_enforce_random:
        random_start = 0
        random_end = 5000
        random_target = random_start + (random_end - random_start) / 2
    
    year_map, case_list = get_year_map(filename)
    
    # Begin building citation graph
    f = get_file_obj(filename)    
    g = nx.DiGraph()
    
    for line in f:
    
        year, from_case, to_case = line[:-1].split(',')

        # Citing a future case is invalid
        if (to_case in year_map) and (from_case in year_map) and (year_map[from_case] < year_map[to_case]):
            continue
            
        # If they cite each other, it is most likely that both are a wrong edge
        if (to_case in g) and (from_case in g[to_case]):
            del g[to_case][from_case]
            continue
        
        # Add edges
        if (b_enforce_random == False) or (random_target == random.randrange(random_start, random_end)):
            g.add_edge(from_case, to_case)
#             g.node[from_case]['year']=int(year)
            g.node[from_case]['ifCounted']=False
            g.node[from_case]['isMemer']=False
            g.node[from_case]['citesMemer']=False
            g.node[to_case]['ifCounted']=False
            g.node[to_case]['isMemer']=False
            g.node[to_case]['citesMemer']=False
                
    f.close()
    
    return g
    

def display_graph(graph):
    
    print graph.nodes()


def topological_sort(graph):
    """
    Sorts graph by year.
    Argument: graph
    Returns: a list of nodes of the graph sorted by dependencies
    """
    
    global g_filename
    
    if g_filename is None:
        return graph.nodes()
    
    year_map, case_list = get_year_map(g_filename)
    
    return case_list

def freshen_graph(g):
    for node in g.nodes():
        g.node[node]['ifCounted']=False
        g.node[node]['isMemer']=False
        g.node[node]['citesMemer']=False
    return g

In [42]:
def gramScorer(gram,sorted_list,graph,gram_dict,verbose=0):
    """
    Calculates meme score for each ngram.
    Arguments:
    gram: an n-gram or phrase for which meme score to be calculated
    sorted_list: a sorted list of all nodes in graph
    graph: a NetworkX DAG of nodes/cases with edges
    gram_dict: dict storing n-grams
    Returns:
    meme_score: final meme_score of the n-gram
"""
    graph_size = len(sorted_list) #number of nodes
    if verbose>=1:
        print "graph_size is",graph_size
    dm2m=0
    d2m=0
    dm2n=0
    d2n=0
    memers=0
    nonmemers=0
    
    #every node stores 3 vars - isMemer, citesMemer, ifCounted (in sum)
    
    for node in sorted_list: #list of nodes sorted topologically
        
        if node not in graph:
            continue
        
        if not graph.node[node]['ifCounted']: #if not counted in sum
            
            if verbose>=3:
                print node,graph.node[node]
                
            if node in gram_dict.keys():

                if gram_dict[node].has_key(gram): #if gram present
                    memers+=1 #has meme
                    graph.node[node]['isMemer']=True

                else:
                    nonmemers+=1

            graph.node[node]['ifCounted']=True


            if len(graph.successors(node))>0: #has children (citers)
                for child in graph.successors_iter(node):

                    if graph.node[child]['isMemer']==True: #soft-check if child has gram
                        graph.node[node]['citesMemer']=True
                    elif gram_dict.has_key(child) and gram_dict[child].has_key(gram): #hard-check using dictionary
                        graph.node[child]['isMemer']=True
                        graph.node[node]['citesMemer']=True
                        break
#                     else:
#                         nonmemers+=1

            if verbose>=3:
                print node,graph.node[node]
            if graph.node[node]['isMemer']==True and graph.node[node]['citesMemer']==True:
                dm2m+=1 #memer cites memer

            elif graph.node[node]['isMemer']==True and graph.node[node]['citesMemer']==False:
                if len(graph.successors(node))!=0: #make sure childless nodes not counted
                    dm2n+=1 #memer cites non-memers

            if graph.node[node]['citesMemer']==True:
                d2m+=1 #cites memers

            elif graph.node[node]['citesMemer']==False:
                if len(graph.successors(node))!=0: #make sure childless nodes not counted
                    d2n+=1 #cites non-memers
        elif verbose>=2:
                print str(node)+" is counted."
    
#     assert memers+nonmemers==graph_size #sanity check for memer size
    if verbose>=1:
        print "memers =",memers
        print "nonmemers =",nonmemers
        print "dm2m =",dm2m
        print "d2m =",d2m
        print "dm2n =",dm2n
        print "d2n =",d2n

    prop_score = (float(dm2m)/(d2m+3.))/((dm2n+3.)/(d2n+3.)) #ratio of sticking factor by sparking factor
    #we take a shifted prop. score to make sure score not infinite
    freq = float(memers)/(graph_size) #ratio of # of memers for that n-gram to size of graph
    meme_score = prop_score*freq #prop_score x meme frequency
    
    if verbose>=1:
        print "prop_score =",prop_score
        print "freq =",freq
        print "meme_score =",meme_score

    return meme_score

In [17]:
# data_file = 'data/graph_stripped.csv'
data_file = 'data/graph_mini.csv'

In [18]:
%%time
g = get_graph(data_file)
print "total # of nodes: ",len(g.nodes())
print "total # of edges: ",len(g.edges())

# g = eliminate_cycles(g)
# print "total # of non-cyclic nodes: ",len(g.nodes())
# print "total # of non-cyclic edges: ",len(g.edges())

total # of nodes:  130502
total # of edges:  444130
CPU times: user 4.94 s, sys: 148 ms, total: 5.09 s
Wall time: 5.09 s


In [19]:
sorted_list = topological_sort(g)

In [22]:
print "num of cases is: ", len(sorted_list)

num of cases is:  594822


In [28]:
data_dir='data/n_grams/'

In [38]:
def find_files(directory, pattern):
    for root, dirs, files in os.walk(directory):
        for basename in files:
            
            if basename == '.DS_Store':
                continue
            
            if fnmatch.fnmatch(basename, pattern):
                filename = os.path.join(root, basename)
                yield filename

In [40]:

gram_dict={}
#test=['XFKEIQ.txt','XFKK8M.txt','XFLD3P.txt']

#Generate dictionary of n-grams



# for filename in os.listdir(data_dir):
for filename in find_files(data_dir, '*'):
    f = open(filename, 'r')
    name = filename[:-4]
    gram_dict[name]={}
    
    for line in f:
        text, count = line.rsplit(',',1)
        count=int(count[:-2])
        
        if count>1:
            gram_dict[name][text]=count
            
    f.close()

### Sequential version

In [44]:
%%time
testset=gram_dict.keys()
score_dict={}

for i in testset[:1]: #testset of nodes
    for gram in gram_dict[i]:
        if not score_dict.has_key(gram) and gram_dict[i][gram]>=2: #not already scored, freq >= 2
            g = freshen_graph(g) #set all values to False
            gramScore = gramScorer(gram,sorted_list,g,gram_dict,1)
            score_dict[gram] = (i,gramScore) #stores first node encountered and meme score
            
            if int(gramScore)!=0: #only print those with non-zero scores
                print i,gram,gramScore

graph_size is 594822
memers = 0
nonmemers = 0
dm2m = 0
d2m = 0
dm2n = 0
d2n = 102869
prop_score = 0.0
freq = 0.0
meme_score = 0.0
graph_size is 594822
memers = 0
nonmemers = 0
dm2m = 0
d2m = 0
dm2n = 0
d2n = 102869
prop_score = 0.0
freq = 0.0
meme_score = 0.0
graph_size is 594822


KeyboardInterrupt: 

In [35]:
score_dict

{'I have': ('XFVA4E', 0.0),
 'It is': ('XFLCRA', 0.0),
 'It is insisted': ('XFLCRA', 0.0),
 'N. Y': ('XFLCRA', 0.0),
 'Power and Kindred': ('XFLCRA', 0.0),
 'We are': ('XFLCRA', 0.0),
 'We think': ('XFLCRA', 0.0),
 'acquiesced in': ('XFLCRA', 0.0),
 'advised of': ('XFLCRA', 0.0),
 'appoint an assignee': ('XFVA4E', 0.0),
 'are not': ('XFLCRA', 0.0),
 'are of': ('XFLCRA', 0.0),
 'are of opinion': ('XFLCRA', 0.0),
 'as such agents': ('XFLCRA', 0.0),
 'at least two-thirds': ('XFKMOB', 0.0),
 'at such meeting': ('XFKMOB', 0.0),
 'averred that': ('XFLCRA', 0.0),
 'be allowed': ('XFLCRA', 0.0),
 'be pleaded': ('XFLCRA', 0.0),
 'be regarded': ('XFKMOB', 0.0),
 'be regarded as': ('XFKMOB', 0.0),
 'be set': ('XFLCRA', 0.0),
 'been called': ('XFKMOB', 0.0),
 'been called by': ('XFKMOB', 0.0),
 'bill does': ('XFKMOB', 0.0),
 'bill does not': ('XFKMOB', 0.0),
 'bills are': ('XFLCRA', 0.0),
 'bills contain': ('XFLCRA', 0.0),
 'board of directors': ('XFKMOB', 0.0),
 'called by': ('XFKMOB', 0.0),
 'co

### Parallel version 1: grams parallelized

In [36]:
score_list=[]
dic = {'A':('a',1),'B':('b',2),'C':('c',3)}

In [39]:
score_list.append(['B',('b',1)])

In [46]:
score_list

[['A', ('a', 1)], ['B', ('b', 1)]]

In [ ]:

def parallel_scorer(gram):
    score_list=[]
    h=g.copy() #make copy of graph since we overwrite g
    h = freshen_graph(h) #set all values to False
    topo_h=topological_sort(h)
    gramScore = gramScorer(gram,topo_h,h,gram_dict,0)
    #score_dict[gram] = (i,gramScore) #stores first node encountered and meme score
    score_list.append([gram,(i,gramScore)])
    if gramScore!=0: #only print those with non-zero scores
        print i,gram,gramScore
    return score_list




gram_checker={}
all_scores={}
batch_scores=[]

if not gram_checker.has_key(gram) and gram_dict[i][gram]>=2: #not already scored, freq >= 2
    gram_checker[gram]=1 #done
    
    for i in testset[:10]:
        jobs=Parallel(n_jobs=num_cores)(delayed(parallel_scorer)(gram) for gram in gram_dict[i])
        batch_scores.append(jobs)

for x in batch_scores:
    for y in x:
        all_scores[y[0]]=y[1] #all_scores['ngram'] = ('case_id',score)


In [ ]:
with open("gram_scores.csv", "wb") as f:
    csv.writer(f).writerows((k,) + v for k, v in score_dict.iteritems())

In [ ]:
num_cores = multiprocessing.cpu_count()
print "num_cores is: ",num_cores
print "parallel jobs started"
jobs=Parallel(n_jobs=num_cores)(delayed(do_to_case)(case) for case in caseList)
bigout=[]
bignew=pd.DataFrame()
print "parallel jobs done"
print "concatenating df's and out's"
for x in jobs:
    if bignew.empty:
        bignew=x[0]
    else:
        bignew=pd.concat([bignew,x[0]],ignore_index=True)
    bigout = bigout + x[1]
    gc.collect()

batch = 25
for i in range(batch):
    with open("gram_scores_"+str(batch) ".csv", "wb") as f:
        csv.writer(f).writerows((k,) + v for k, v in score_dict.iteritems())

### Parallel version 2: years/cases parallelized

In [52]:
!ls '/scratch/sv1239/projects/mlcs/raw/citation_data/'

1880_complete  1884_complete  1888_complete  1892_complete  1896_complete
1881_complete  1885_complete  1889_complete  1893_complete  1897_complete
1882_complete  1886_complete  1890_complete  1894_complete  1898_complete
1883_complete  1887_complete  1891_complete  1895_complete  1899_complete


In [ ]:
data_dir='/scratch/sv1239/projects/mlcs/raw/'

In [ ]:
def parent_scorer(dirname):
    
    gram_dict={}
    #test=['XFKEIQ.txt','XFKK8M.txt','XFLD3P.txt']

    #Generate dictionary of n-grams

    for filename in os.listdir(data_dir):
        f = open(data_dir+filename, 'r')
        name = filename[:-4]
        gram_dict[name]={}

        for line in f:
            text, count = line.rsplit(',',1)
            count=int(count[:-2])

            if count>1:
                gram_dict[name][text]=count

        f.close()
        


### Part 1: creating gram-dicts

In [ ]:
def get_gram_dict(start_year,end_year):
    """
    saves a .csv file containing case_id x gram x freq. data of all id's from start to end year.
    """
    return

### Part 2a: inner parallel loop over gram-dicts (modified meme-scorer)

### Part 2b: outer loop over cases in a year, and grams in a case

### part 3: outer outer parallel loop over years

In [ ]:
def parallel_scorer_by_year(gram):
    #same as parallel_scorer
    return score_list

def file_list(directory):
    """
    Returns list containing names of files in directory. Will parallelize over these.
    """


In [57]:
# step 1: generate gramdicts for each decade (for size).
# step 2: modify meme_scorer func so it stores dm2m, d2m etc. This is calculated parallely over all gramdicts.
# step 3: sum up all dm2m etc. to get final meme_score for each gram.
# step 3.5: loop over grams in a case for a year
# step 4: loop over cases in a year
# step 5: outer outer parallel loop over cases in parallelized years

# 5-4-3.5-3-2-1.

# 2 parallelizations - 1) over years (folders) 2) over gram_dict for score calculation.
